In [1]:
import numpy as np
import pandas as pd

import re # regular expression
import os # operating system
import shutil # shell utilities
import fnmatch # file name matching
import warnings;warnings.filterwarnings("ignore")

Count lines like

defined( GEOS_FP ) || defined( MERRA2 ) 

or #if !defined( MERRA ) && !defined( GEOS_FP )

In [5]:
maindir='../Code_v11-02_FlexGrid'

Tags = ['NESTED_NA','NESTED_CH','NESTED_CU']

# match something like #if defined( GEOS_FP ) or #elif defined ( MERRA2)
# white spaces can occur at many places! Use ( )* to represent them
regex_list = [re.compile(r'#(.*)' +
                         r'defined( )*'+
                         r'\(( )*' +
                         Tag + 
                         r'( )*\)' ) 
              for Tag in Tags]  
n_tag = len(Tags)

fileFormat = '*.F*' # .F or .F90

data = [] # a list of list, to be converted to pandas DataFrame
for dirpath, dirs, files in os.walk(maindir):

    for filename in fnmatch.filter(files, fileFormat):
    
        # contruct the full path
        filepath = os.path.join(dirpath, filename)
        
        # KPP/*/gckpp_HetRates.F90 are mostly links, skip them
        if os.path.islink(filepath):
            continue
        
        # record current file, followed by n_tag+2 0s for later use
        data.append( [dirpath[len(maindir):],filename] + [0]*(n_tag+2) )
        
        # read the file line by line
        with open(filepath,"r",encoding = "ISO-8859-1") as file:
            for line in file:
                for n,regex in enumerate(regex_list):          
                    if regex.match(line):
                        data[-1][n+2] += 1
                        #print(line)
                if regex_list[0].match(line) and regex_list[1].match(line):
                    if "||" in line:
                        data[-1][n_tag+2] += 1
                        #print(line)
                    if "&&" in line:
                        data[-1][n_tag+3] += 1
                        #print(line)

#label the raw data
df_raw = pd.DataFrame(data, columns=['dir','filename']+Tags+['NA || CU','NA !&& CH'])

#filter the files with zero match, and re-arrange the index
df = df_raw[(df_raw['NESTED_NA'] > 0) | (df_raw['NESTED_CH'] > 0)] 
df.reset_index(drop=True,inplace=True)
 
# display results
df

,dir,filename,NESTED_NA,NESTED_CH,NESTED_CU,NA || CU,NA !&& CH
0,/GeosCore,dao_mod.F,1,1,0,1,0
1,/GeosCore,exchange_mod.F,2,2,0,1,0
2,/GeosCore,geosfp_read_mod.F90,1,1,0,0,0
3,/GeosCore,input_mod.F,5,4,0,1,0
4,/GeosCore,main.F,2,2,0,2,0
5,/GeosCore,mercury_mod.F,2,0,0,0,0
6,/GeosCore,merra2_read_mod.F90,1,0,0,0,0
7,/GeosCore,seasalt_mod.F,0,1,0,0,0
8,/GeosCore,strat_chem_mod.F90,5,4,0,2,0
9,/GeosCore,tpcore_bc_mod.F,5,5,1,1,0
